0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [331]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [330]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [383]:
y = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

In [385]:
y = y.iloc[:,[1]]

In [386]:
y

,0
0,5.0
1,6.0
2,7.0
3,8.0
4,7.0
...,...
14271,5.0
14272,8.0
14273,2.0
14274,6.0


In [387]:
y_100_rows = y[0:100]

In [391]:
y_arr = y_100_rows.to_numpy()

In [5]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis')

NEW APPROACH:

Fit CNN on X-Array. Problem: How to get RGB from array?
**SOLVED**. See Cell below ("GET RGBA VALUES FOR EACH HEATMAP").

RGB NOT NECCESSARY FOR CNN. STILL ADDING THIRD DIMENSION?

1. Read first 10 rows from X.

In [338]:
chunk = 10
i=0
X=[]
import pandas as pd
for df in pd.read_csv('X_matrix.csv',chunksize = chunk):
    X.append(df)
    i=i+1
    print('Processed {i}/10 chunks'.format(i=i))
    if i == 10:
        break



Processed 1/10 chunks
Processed 2/10 chunks
Processed 3/10 chunks
Processed 4/10 chunks
Processed 5/10 chunks
Processed 6/10 chunks
Processed 7/10 chunks
Processed 8/10 chunks
Processed 9/10 chunks
Processed 10/10 chunks


In [342]:
X = pd.concat(X)

In [344]:
X_arr = X.to_numpy()

Adding new dimension for CNN

In [449]:
X_ = X_arr[:,:,np.newaxis]

In [ ]:
X_ = np.swapaxes(X_,2,1)

In [450]:
X_.shape

(100, 50177, 1)

Converting X to appropriate Datatype...

In [454]:
X_tensor_train = torch.from_numpy(X_)
y_tensor_train = torch.from_numpy(y_df)

In [455]:
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
y_tensor_train = y_tensor_train.type(torch.LongTensor)

/tmp/ipykernel_790/1671349079.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)


In [456]:
X_tensor_train.shape

torch.Size([100, 50177, 1])

In [407]:
y_tensor_train.shape

torch.Size([100, 1])

Setup transformer

In [408]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y


Setting up the CNN.

In [548]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 12544, 128)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        #x = torch.flatten(x, 1)
        #x = x.view(x.size(0), -1)
        x = x.view(-1, 32 * 12544)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x


In [549]:
c = CNN()

In [431]:
import torch.nn as nn
import torch.nn.functional as F

class saliencnn(nn.Module):
    def __init__(self):
        super(saliencnn, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool1d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [432]:
CNN = saliencnn()

In [486]:
X_tensor_train.shape

torch.Size([100, 50177, 1])

In [487]:
#XXX = X_tensor_train.permute(0,2,1)
#XXX.shape

torch.Size([100, 1, 50177])

In [579]:
#XXX.shape

torch.Size([100, 1, 50177])

In [ ]:
dataset = CustomDataset(X_tensor_train,y_tensor_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [576]:
def train(net, loss_fn, dataloader,
          n_epochs=50, learning_rate=0.01):
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate,weight_decay=1e-4)
    train_loss = []
    
    for i in range(n_epochs):
        net.train()
        for data, label in dataloader:
        #make prediction
            output = net(data)

            # Zero out the gradients
            optimizer.zero_grad()

            # Calculate loss.
            loss = criterion(y,torch.max(label, 1)[1])

            # Backpropagation and gradient update.
            loss.backward(retain_graph=True)# Calculate gradients.

            optimizer.step() # Apply gradient udpate.

            train_loss.append(loss.item())

            if (i + 1) % (n_epochs // 5) == 0:
                print(f'iteration {i + 1}/{n_epochs} | loss: {loss.item():.3f}')
        y_pred = torch.softmax(out,dim=1).argmax(dim=1)


In [577]:
train(c,criterion,dataloader)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [578]:
dataset = CustomDataset(XXX,y_tensor_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(c.parameters(), lr=0.001)
train_loss = []

for epoch in range(5):
    c.train()
    for data, label in dataloader:
        #make prediction
        output = c(data)

        # Zero out the gradients
        optimizer.zero_grad()
            
        # Calculate loss.
        loss = criterion(y, torch.max(label, 1)[1])
            
        # Backpropagation and gradient update.
        loss.backward(retain_graph=True) # Calculate gradients.
            
        optimizer.step() # Apply gradient udpate.
        
        train_loss.append(loss.item())
        
    if (i + 1) % (n_epochs // 5) == 0:
        print(f'iteration {i + 1}/{n_epochs} | loss: {loss.item():.3f}')
    y_pred = torch.softmax(out,dim=1).argmax(dim=1)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.